In [2]:
import os
import json
import pandas as pd
from datetime import datetime

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

json_dir = "ipl_json"
matches_info = []

In [13]:
def extract_match_info(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
        info = data['info']
        match_date = info['dates'][0]
        match_id = f"{data['info']['dates'][0]}_{data['info']['teams'][0]}_{data['info']['teams'][1]}"
        city = data['info'].get('city', 'Dubai')
        season = data['info']['season']
        team1, team2 = data['info']['teams']
        toss_winner = data['info']['toss']['winner']
        toss_decision = data['info']['toss']['decision']
        outcome = data['info'].get('outcome', {})
        result = outcome.get('result', 'normal') 
        
        if 'winner' in outcome:
            winner = outcome['winner']
        elif 'eliminator' in outcome:
            winner = outcome['eliminator']
        else:
            winner = None
        
        player_of_match = data['info'].get('player_of_match', [None])[0]
        venue = data['info']['venue']
        umpires = data['info']['officials']['umpires']
        umpire1, umpire2 = umpires[:2]
        
        method_dl = 1 if 'method' in outcome and outcome['method'] == 'D/L' else 0
        
        win_by_runs = outcome.get('by', {}).get('runs', 0)
        win_by_wickets = outcome.get('by', {}).get('wickets', 0)
        
        return match_date, season, city, team1, team2, toss_winner, toss_decision, method_dl, result, winner, win_by_runs, win_by_wickets, player_of_match, venue, umpire1, umpire2, match_id, 

matches_info = []

for filename in os.listdir(json_dir):
    if filename.endswith(".json"):
        json_file_path = os.path.join(json_dir, filename)
        match_info = extract_match_info(json_file_path)
        matches_info.append(match_info)

matches_info.sort(key=lambda x: x[0], reverse=True)

matches_df = pd.DataFrame(matches_info, columns=['Match Date', 'Season', 'City', 'Team 1', 'Team 2', 'Toss Winner', 'Toss Decision', 'Dl applied', 'Result', 'Winner', 'Won by Runs', 'Won by Wickets', 'Player of the Match', 'Venue', 'Umpire 1', 'Umpire 2', 'Match Id'])


matches_df.to_csv('matches_info.csv', index=False)
